In [8]:
import numpy as np
from omegaconf import OmegaConf, DictConfig
from kitti360 import HIL_rendering, custom_run_window_config, DEFAULT_ARGS
import moderngl_window as mglw
BASE_KITTI_CONFIG_PATH = 'configs/kitti360.yaml'

In [13]:
def gen_random_actor_cfg(n_actors: int):
    cfg = {}
    for i in range(n_actors):
        id = 'actor_' + repr(i)
        anim_types = [ 'man_running', 'woman_walking' ] # folder name of the asset from ./assets/human_animations
        animation_type =  str(np.random.choice(anim_types).item())
        x_val = np.random.uniform(-10.0, 5.0) # ranges based on kitti sidewalk limits
        y_val = 0.0 # how high above ground
        z_val = np.random.uniform(4.0, 12.0) # how far away from ego vehicle
        pos = [x_val, y_val, z_val]
        if x_val < -2.5: # actor to the left of the car --> make angle so that actor approximately faces right on average
            rot_angle = np.random.uniform(30, 150)
        else:
            rot_angle = np.random.uniform(-150, -45)
        rot = [0.0, rot_angle, 0.0]
        scale = [2.5, 2.5, 2.5]
        cfg[id] = {
                'animation_type': animation_type,
                'object2scenario': {
                    'pos' : pos,
                    'rot' : rot,
                    'scale' : scale
                }
            }
    return cfg
def auto_gen_kitti_config() -> DictConfig:
    base_cfg = OmegaConf.load(BASE_KITTI_CONFIG_PATH)
    actor_gen = gen_random_actor_cfg(np.random.randint(1, 3))
    base_cfg.scenario_configs.jaywalking.actor_configs = actor_gen
    base_cfg.scenario_configs.jaywalking.trigger_distance = 2.0 # just force immediate animation with low trigger distance
    print(base_cfg.scenario_configs.jaywalking.actor_configs)
    base_cfg.device = 'cpu'
    base_cfg.display = 'headless'
    return base_cfg

In [14]:
custom_config = auto_gen_kitti_config()
config_cls = HIL_rendering
# parser = mglw.create_parser()
# config_cls.add_arguments(parser)
# values = mglw.parse_args(args=None, parser=parser)
config_cls.argv = DEFAULT_ARGS
values = DEFAULT_ARGS
custom_run_window_config(custom_config, config_cls, values)

2023-08-23 08:19:51,387 - moderngl_window - INFO - Attempting to load window class: moderngl_window.context.headless.Window
2023-08-23 08:19:51,431 - moderngl_window.context.base.window - INFO - Context Version:
2023-08-23 08:19:51,432 - moderngl_window.context.base.window - INFO - ModernGL: 5.7.2
2023-08-23 08:19:51,433 - moderngl_window.context.base.window - INFO - vendor: NVIDIA Corporation
2023-08-23 08:19:51,433 - moderngl_window.context.base.window - INFO - renderer: NVIDIA GeForce RTX 4050 Laptop GPU/PCIe/SSE2
2023-08-23 08:19:51,434 - moderngl_window.context.base.window - INFO - version: 3.3.0 NVIDIA 532.09
2023-08-23 08:19:51,434 - moderngl_window.context.base.window - INFO - python: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
2023-08-23 08:19:51,435 - moderngl_window.context.base.window - INFO - platform: win32
2023-08-23 08:19:51,435 - moderngl_window.context.base.window - INFO - code: 330


{'actor_0': {'animation_type': 'man_running', 'object2scenario': {'pos': [2.7685745266706068, 0.0, 6.09861906736138], 'rot': [0.0, -145.53307290632404, 0.0], 'scale': [2.5, 2.5, 2.5]}}}
frame 2, FPS = 8.10839798252007, time: 0.2466578483581543
frame 3, FPS = 6.003718772261222, time: 0.49969029426574707
frame 4, FPS = 5.282347388661093, time: 0.7572391033172607
frame 5, FPS = 4.917547360497863, time: 1.0167670249938965
frame 6, FPS = 4.710989640467695, time: 1.2736177444458008
frame 7, FPS = 4.579889474770394, time: 1.52842116355896
frame 8, FPS = 4.487936802146114, time: 1.7825562953948975
frame 9, FPS = 4.415715325969662, time: 2.038174867630005
frame 10, FPS = 4.364574791776796, time: 2.2911739349365234
frame 11, FPS = 4.330409559812087, time: 2.540175437927246
frame 12, FPS = 4.296185533826467, time: 2.793175458908081
frame 13, FPS = 4.269417838533867, time: 3.0449116230010986
frame 14, FPS = 4.250330951821709, time: 3.293861150741577
frame 15, FPS = 4.230926903274746, time: 3.54532

2023-08-23 08:20:00,682 - moderngl_window - INFO - Duration: 5.35s @ 4.11 FPS


frame 21, FPS = 4.133183660931759, time: 5.08082914352417
Cleanup
